<a href="https://colab.research.google.com/github/kuuuun/python_jupyter_notes/blob/main/chaptor05_ufuncs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# chaptor05 Ufuncs